<a href="https://colab.research.google.com/github/ravilon/ufpel-pdi/blob/main/LAB6_Remocao_Ruido_Melhoria_Qualidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB6 — Pipeline de Processamento para Remoção de Ruído e Melhoria de Qualidade Visual

Disciplina: **Processamento Digital de Imagens (PDI)** – UFPel  
Professor: **Guilherme Corrêa**  

Vamos exercitar os conceitos estudados até o momento criando um pipeline de processamento de imagens.

Um pipeline de PDI é um fluxo com diversas operações executadas sequencialmente. No caso deste laboratório, vamos executar diversas operações para tentar melhorar a qualidade de uma imagem que tem baixa resolução, sofreu efeito de ruído periódico e também sofreu efeito de ruído sal & pimenta.

Use a imagem `tuyuka_lq.png` está disponível no repositório para os exercícios.

---

## Objetivos  

1. Aplicar conceitos estudados até este momento para melhorar a qualidade de uma imagem que tem baixa resolução, sofreu efeito de ruído periódico e também sofreu efeito de ruído sal & pimenta.
2. Você deve escolher quais passos de PDI aplicar e em qual ordem, considerando tudo o que estudamos até aqui (interpolação, realce, suavização, filtragem no domínio espacial, filtragem no domínio de transformadas...
3. **IMPORTANTE: a ordem de aplicação das técnicas vai influenciar muito no resultado final! Escolha sabiamente!**

---

## Bibliotecas úteis
Se estiver no Colab, rode a célula de instalação uma única vez.

In [1]:
# Se necessário no Colab, descomente a linha abaixo:
#!pip -q install numpy matplotlib scikit-image imageio

In [2]:
# (execute uma vez)
!pip -q install ipywidgets==8.1.2 scikit-image==0.24.0 opencv-python-headless==4.10.0.84
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
import matplotlib.pyplot as plt
import cv2
from ipywidgets import interact, IntSlider
from skimage import filters, img_as_float
from skimage import io, color, img_as_ubyte
from google.colab import files



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.4 MB/s eta 0:00:00


## Upload de uma imagem
Usando `files` do `google.colab` para fazer upload de uma imagem.

In [ ]:

print("Faça upload de uma imagem (JPG/PNG).")
up = files.upload()
if not up:
    raise RuntimeError("Nenhum arquivo enviado.")

# Nome do arquivo
fname = next(iter(up))

# Ler imagem colorida (BGR)
img_bgr = cv2.imdecode(np.frombuffer(up[fname], np.uint8), cv2.IMREAD_COLOR)
if img_bgr is None:
    raise RuntimeError("Falha ao ler a imagem.")

# Converter para escala de cinza (uint8, faixa 0–255)
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
img_orig = img_gray.astype(np.uint8)

# Mostrar
plt.imshow(img_orig, cmap='gray', vmin=0, vmax=255)
plt.axis('off')
plt.title('Imagem (grayscale, 0–255)')
plt.show()

print("Dimensão:", img_orig.shape, " | dtype:", img_orig.dtype, " | Faixa:", (img_orig.min(), img_orig.max()))


Faça upload de uma imagem (JPG/PNG).


# 🖼️ Passo 1

...



In [ ]:
...

# 🖼️ Passo 2
...

In [ ]:
...